- Medium
- 看Solution里的图, https://leetcode.com/problems/construct-binary-search-tree-from-preorder-traversal/solution/

In [1]:
from collections import deque

# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None
        
    def draw(self):
        queue = deque([self])
        while queue:
            nxt_queue = deque([])
            while queue:
                node = queue.popleft()
                if node is None:
                    print('None', end=' ')
                    nxt_queue.extend([None, None])
                else:
                    print(node.val, end=' ')
                    nxt_queue.extend([node.left, node.right])
            print()
            if all([i is None for i in nxt_queue]):
                nxt_queue = deque([])
            queue = nxt_queue

- 通过preorder和inorder: preorder能确定root在哪, inorder能确定left, right分别有什么
- 逻辑上是遍历preorder, 然后在inorder内找该值的index, 然后就能递归, left = helper(l, index), right = helper(index+1, r)
- TC=O(N*logN), 因为要排序成inorder, 所以不是最优解

In [19]:
class Solution:
    def bstFromPreorder(self, preorder):
        
        def helper(l, r):
            nonlocal preorder_index
            if l == r:
                return None
            
            cur_val = preorder[preorder_index]
            cur_node = TreeNode(cur_val)
            inorder_index = idx_map[cur_val]
            preorder_index += 1
            cur_node.left = helper(l, inorder_index)
            cur_node.right = helper(inorder_index+1, r)
            return cur_node
            
        inorder = sorted(preorder)
        idx_map = {e: i for i, e in enumerate(inorder)}
        preorder_index = 0
        return helper(0, len(preorder))

In [20]:
s = Solution()
s.bstFromPreorder([8,5,1,7,10,12]).draw()

8 
5 10 
1 7 None 12 


- recursion
- 可以通过(lower, upper)给每个subtree限定范围, 那如果当前的val不在这范围内, 就说明不是这个subtree的root, 直接返回None
- TC=O(N)

In [ ]:
class Solution:
    def bstFromPreorder(self, preorder: List[int]) -> TreeNode:
        def helper(lower, upper):
            nonlocal idx
            # 遍历完了
            if idx == n:
                return None
            
            val = preorder[idx]
            # 该node不在此subtree内
            if val < lower or val > upper:
                return None
            
            idx += 1  # 当前Node放好了, 看下一个
            root = TreeNode(val)
            root.left = helper(lower, val)
            root.right = helper(val, upper)
            return root
            
        idx = 0
        n = len(preorder)
        return helper(float('-inf'), float('inf'))

- Iteration
- 用stack记录没完全处理完的node. 完全处理完是指 1). 其left subtree遍历完了, 剩下的都是比它大的nodes, 要么是它的right subtree, 要么就是它某个父节点的right subtree, 2). 无论哪种情况, 都可以从stack里pop了, 因为如果是其right subtree, 那当前node的left和right都已确定, 接下来要用的也是该right subtree的root, 而如果是其父节点的, 那就一直pop到其父节点, 然后也就一个道理了
- TC=O(N)

In [ ]:
class Solution:
    def bstFromPreorder(self, preorder: List[int]) -> TreeNode:
        n = len(preorder)
        if n == 0:
            return []
        
        root = TreeNode(preorder[0])
        stack = [root]
        
        for i in range(1, n):
            node = stack[-1]  # stack顶上当做parent
            child = TreeNode(preorder[i])
            
            # child没上一个大, 那说明就是在left subtree
            if node.val >= child.val:
                node.left = child
            # else就说明left subtree遍历完了, 开始看right subtree ...
            else:
                # ... 但其实并不确定是前面哪个node的right subtree, 所以要while, 直到child比当前node大, 但又没上一个node大 ...
                while stack and stack[-1].val < child.val:
                    node = stack.pop()
                # ... 就说明这child是node的right subtree
                node.right = child
            
            stack.append(child)
            
        return root